In [1]:
import os

In [2]:
%pwd

'd:\\souvik projects\\End-to-End-Chest-cancer-classification-project-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\souvik projects\\End-to-End-Chest-cancer-classification-project-'

In [5]:
from dataclasses import dataclass   
from pathlib import Path 
from typing import List



@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([Path(self.config["artifacts_root"])])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config["data_ingestion"]

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config["root_dir"]),
            source_URL=config["source_URL"],
            local_data_file=Path(config["local_data_file"]),
            unzip_dir=Path(config["unzip_dir"])
        )

        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from CNN_Classifier import logger
from CNN_Classifier.utils.common import get_size    

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,str(zip_download_dir))

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)  


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-10-30 07:41:59,031 :  INFO : common] : yaml file: config\config.yaml loaded successfully]


[2025-10-30 07:41:59,054 :  INFO : common] : yaml file: params.yaml loaded successfully]
[2025-10-30 07:41:59,064 :  INFO : common] : created directory at: artifacts]
[2025-10-30 07:41:59,067 :  INFO : 2712197382] : Downloading data from https://drive.google.com/file/d/1f0KyaqPa3UeDSRN1TR_RnVL9WCTwIb10/view?usp=sharing into file artifacts\data_ingestion\data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1f0KyaqPa3UeDSRN1TR_RnVL9WCTwIb10
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1f0KyaqPa3UeDSRN1TR_RnVL9WCTwIb10&confirm=t&uuid=cb261ff1-a5d5-487b-ac6e-fb8b6db59ae0
To: d:\souvik projects\End-to-End-Chest-cancer-classification-project-\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:08<00:00, 6.08MB/s]

[2025-10-30 07:42:11,631 :  INFO : 2712197382] : Downloaded data from https://drive.google.com/file/d/1f0KyaqPa3UeDSRN1TR_RnVL9WCTwIb10/view?usp=sharing into file artifacts\data_ingestion\data.zip]
